In [1]:
def mymodel(act, units):
    text_in = Input(shape=(768,),name='text_input')#[768]
    img_in = Input(shape=(512,),name='image_input')#[512]
    
    
    avg_pool_text = Dense(1)(text_in)
    avg_pool_image = Dense(1)(img_in)
    
    concate = Concatenate(axis=1)([avg_pool_text, avg_pool_image])
    fc_1 = Dense(1, activation='relu')(concate)
    fc_2 = Dense(2, activation='sigmoid')(fc_1)
    fc_2 = Reshape((2, 1))(fc_2)
    
    
    concate_feature = Concatenate(axis=1)([text_in, img_in])#1280
    # concate_feature = Dropout(0.1)(concate_feature)
    concate_feature = Reshape((2, 640))(concate_feature)
    

    dot_result = Dot(axes=1)([fc_2, concate_feature])
    
    dot_result = Reshape((640,))(dot_result)
    
    # dot_result = Dense(512)(dot_result)
    # dot_result = Dense(256)(dot_result)
    # dot_result = Dense(128)(dot_result)
    dot_result = Dense(64)(dot_result)
    dot_result = Dense(32)(dot_result)
    
    
    output = Dense(units,activation=act)(dot_result)
    model = Model([text_in, img_in], output)
    return model

In [2]:
from keras import backend as K

def categorical_crossentropy_with_prior(y_true, y_pred, tau=1.0):
    """带先验分布的交叉熵
    注：y_pred不用加softmax
    """
    prior = [0.25,0.42,0.33]  # 自己定义好prior，shape为[num_classes]
    log_prior = K.constant(np.log([i + 1e-8 for i in prior]))
    for _ in range(K.ndim(y_pred) - 1):
        log_prior = K.expand_dims(log_prior, 0)
    y_pred = y_pred + tau * log_prior
    return K.categorical_crossentropy(y_true, y_pred, from_logits=True)

def sparse_categorical_crossentropy_with_prior(y_true, y_pred, tau=1.0):
    """带先验分布的稀疏交叉熵
    注：y_pred不用加softmax
    """
    prior = [0.21, 0.28, 0.43, 0.08]    # 自己定义好prior，shape为[num_classes][0.25,0.42,0.33]  [0.14,0.86] [0.21,0.79] [0.61,0.39][0.88,0.12]
    #[0.14, 0.48, 0.29, 0.08] [0.21, 0.28, 0.43, 0.08][0.61, 0.28, 0.09, 0.03]#!!!!!替换点
    log_prior = K.constant(np.log([i + 1e-8 for i in prior]))
    for _ in range(K.ndim(y_pred) - 1):
        log_prior = K.expand_dims(log_prior, 0)
    y_pred = y_pred + tau * log_prior
    return K.sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [3]:
from tensorflow.keras.layers import Input, Conv1D, Dropout, GlobalAveragePooling1D, Concatenate, Conv2D, GlobalAveragePooling2D, Dense, Embedding, LSTM, Dot, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.applications.densenet import DenseNet121
import tensorflow.keras.preprocessing.sequence as S
from tensorflow.keras.optimizers import Adam,SGD

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
import pandas as pd

import numpy as np

model = mymodel('softmax', 4)#sigmoid#!!!!!!!替换点
model.compile(loss=sparse_categorical_crossentropy_with_prior, optimizer=Adam(lr=1e-4), metrics=['sparse_categorical_accuracy'])

2022-12-06 09:29:24.594298: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-12-06 09:29:24.594333: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-06 09:29:24.594349: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (xyxyxxxy-tfotywoiijsp-main): /proc/driver/nvidia/version does not exist
2022-12-06 09:29:24.594499: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.8/site-pa

In [4]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text_input (InputLayer)        [(None, 768)]        0           []                               
                                                                                                  
 image_input (InputLayer)       [(None, 512)]        0           []                               
                                                                                                  
 dense (Dense)                  (None, 1)            769         ['text_input[0][0]']             
                                                                                                  
 dense_1 (Dense)                (None, 1)            513         ['image_input[0][0]']            
                                                                                              

In [63]:
train_image_path = 'memotion3features/train_image_feature.csv'
val_image_path = 'memotion3features/val_image_feature.csv'
test_image_path = 'memotion3features/test_image_feature.csv'

train_text_path = 'memotion3features/train_text_feature.csv'
val_text_path = 'memotion3features/val_text_feature.csv'
test_text_path = 'memotion3features/test_text_feature.csv'

train_label_path = 'train.csv'
val_label_path = 'val.csv'


def load_data(text_path, image_path, label_path):
    text_features = pd.read_csv(text_path)
    image_features = pd.read_csv(image_path)
    labels_all = pd.read_csv(label_path)['c2_labels2']#!!!!!!替换点
    
    # features = pd.concat([text_features,image_features],axis=1)
#     print(features.shape)
    
    return [text_features, image_features], labels_all

train_X, train_y = load_data(train_text_path, train_image_path, train_label_path)
val_X, val_y = load_data(val_text_path, val_image_path, val_label_path)

In [13]:
from keras.callbacks import LambdaCallback
from sklearn.metrics import f1_score, confusion_matrix
from keras.callbacks import ModelCheckpoint

filepath='best.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_sparse_categorical_accuracy', verbose=1, save_best_only=True,mode='max',period=1)

batch_print_callback = LambdaCallback(on_epoch_end=lambda batch,\
                    logs: print("train f1:", f1_score(train_y, \
                    model.predict(train_X).argmax(axis=-1), average='weighted'), \
                    "val f1:", f1_score(val_y, model.predict(val_X).argmax(axis=-1),\
                    average='weighted')))

model.fit(train_X,train_y,batch_size=256,epochs=30,
        validation_data=(val_X,val_y),
        callbacks=[batch_print_callback,checkpoint] 
)

Epoch 1/30
17/28 [=================>............] - ETA: 0s - loss: 1.2311 - sparse_categorical_accuracy: 0.3718train f1: 0.3847970789498792 val f1: 0.3890091860878044

Epoch 1: val_sparse_categorical_accuracy improved from -inf to 0.33533, saving model to best.h5
28/28 [==============================] - 0s 16ms/step - loss: 1.2270 - sparse_categorical_accuracy: 0.3750 - val_loss: 1.2002 - val_sparse_categorical_accuracy: 0.3353
Epoch 2/30
16/28 [================>.............] - ETA: 0s - loss: 1.2267 - sparse_categorical_accuracy: 0.3989train f1: 0.38709229526638 val f1: 0.4006778544313788

Epoch 2: val_sparse_categorical_accuracy improved from 0.33533 to 0.34933, saving model to best.h5
28/28 [==============================] - 0s 17ms/step - loss: 1.2261 - sparse_categorical_accuracy: 0.3937 - val_loss: 1.1976 - val_sparse_categorical_accuracy: 0.3493
Epoch 3/30
15/28 [===============>..............] - ETA: 0s - loss: 1.2176 - sparse_categorical_accuracy: 0.3953train f1: 0.389068798

In [64]:
from tensorflow.keras.models import load_model

model_load = load_model('bestmodel/taskC2_0.4700.h5',custom_objects={'sparse_categorical_crossentropy_with_prior': sparse_categorical_crossentropy_with_prior})

In [9]:
from sklearn.metrics import f1_score, confusion_matrix

In [12]:
# confusion_matrix(val_y, predict.argmax(axis=-1))

In [65]:
predict = model_load.predict(val_X)
f1_score(val_y, predict.argmax(axis=-1), average='weighted')

0.4706675292845772